script to extract sub-neurons and correctly align with corresponding image

In [1]:
## RUN THIS CELL FIRST AND WAIT FOR gui qt5 to finish
%gui qt5 
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from skimage import io
import networkx as nx
import napari

In [2]:
## functions to read swc into a pandas dataframe
def read_swc(path):
    """Read a single swc file

    Arguments:
        path {string} -- path to file

    Returns:
        df {pandas dataframe} -- indices, coordinates, and parents of each node
        offset {list of floats} -- offset value of fragment
        color {list of ints} -- color
        cc {int} -- cc value, from file name
        branch {int} -- branch number, from file name
    """

    #check input
    file = open(path,'r')
    in_header = True
    offset_found = False
    header_length = -1
    offset = np.nan
    color = np.nan
    cc = np.nan
    branch = np.nan
    while in_header:
        line = file.readline().split()
        if 'OFFSET' in line:
            offset_found=True
            idx = line.index('OFFSET')+1
            offset = [float(line[i]) for i in np.arange(idx,idx+3)]
        elif 'COLOR' in line:
            idx = line.index('COLOR')+1
            line = line[idx]
            line = line.split(',')
            color = [float(line[i]) for i in np.arange(len(line))]
        elif 'NAME' in line:
            idx = line.index('NAME')+1
            name = line[idx]
            name = re.split('_|-|\.',name)
            idx = name.index('cc')+1
            cc = int(name[idx])
            idx = name.index('branch')+1
            branch = int(name[idx])
        elif line[0] != '#':
            in_header = False
        header_length += 1

    if not offset_found:
        raise IOError('No offset information found in: ' + path)
    #read coordinates
    df = pd.read_table(path,names=['sample','structure','x','y','z','r','parent'],skiprows=header_length,delim_whitespace=True)
    return df, offset, color, cc, branch


def read_swc_offset(path):
    df, offset, color, cc,branch = read_swc(path)
    df['x'] = df['x'] + offset[0]
    df['y'] = df['y'] + offset[1]
    df['z'] = df['z'] + offset[2]

    return df, color, cc, branch

def space_to_voxel(spatial_coord,spacing,origin=np.array([0,0,0])):
    """converts 3D coordinate from spatial coordinates to voxel coordinates
        
        Arguments
            spatial_coord -- coordinate in spatial coordinates
            spacing -- conversion factor (spatial unit/voxel)
            origin -- origin of spatial coordinate
    """
    voxel_coord = np.round(np.divide(spatial_coord - origin, spacing))
    voxel_coord = voxel_coord.astype(int)
    return voxel_coord

def swc_to_voxel(df,spacing,origin=np.array([0,0,0])):
    """converts dataframe representing swc into voxel coordinates
        
        Arguments
            df -- dataframe representing the swc
            spacing -- conversion factor (spatial unit/voxel)
            origin -- origin of spatial coordinate
    """
    x = []; y = []; z = []
    for index, row in df.iterrows():
        vox = space_to_voxel(row[['x','y','z']].to_numpy(),spacing,origin)
        x.append(vox[0])
        y.append(vox[1])
        z.append(vox[2])
    df['x'] = x
    df['y'] = y
    df['z'] = z
    
    return df


In [3]:
## functions for working with neurons as networkx diGraphs
def df_to_graph(df):
    """Converts dataframe of swc in voxel coordinates into a directed graph
    """
    G = nx.DiGraph()

    # add nodes
    for index, row in df.iterrows():
        id = int(row['sample'])

        G.add_node(id)
        G.nodes[id]['x'] = int(row['x'])
        G.nodes[id]['y'] = int(row['y'])
        G.nodes[id]['z'] = int(row['z'])

    # add edges
    for index, row in df.iterrows():
        child = int(row['sample'])
        parent = int(row['parent'])

        if parent > min(df['parent']):
            G.add_edge(parent,child)

    return G

def get_sub_neuron(G, start, end):
    """Returns sub-neuron with node coordinates bounded by start and end"""
    G_cp = G.copy()  # make copy of input G

    # remove nodes that are not neighbors of nodes bounded by start and end
    for node in list(G_cp.nodes):
        neighbors = list(G_cp.successors(node)) + list(G_cp.predecessors(node))

        remove = True

        for id in neighbors + [node]:
            x = G_cp.nodes[id]['x']
            y = G_cp.nodes[id]['y']
            z = G_cp.nodes[id]['z']

            if x >= start[0] and y >= start[1] and z >= start[2]:
                if x < end[0] and y < end[1] and z < end[2]:
                    remove = False

        if remove:
            G_cp.remove_node(node)

    # set origin to start of bounding box
    for id in list(G_cp.nodes):
        G_cp.nodes[id]['x'] = G_cp.nodes[id]['x'] - start[0]
        G_cp.nodes[id]['y'] = G_cp.nodes[id]['y'] - start[1]
        G_cp.nodes[id]['z'] = G_cp.nodes[id]['z'] - start[2]

    return G_cp

In [4]:
## function to process neuron into napari compatible format
def graph_to_paths(G):
    G_cp = G.copy()  # make copy of input G
    branches = []
    while len(G_cp.edges) != 0: #iterate over branches
        # get longest branch
        longest = nx.algorithms.dag.dag_longest_path(G_cp) # list of nodes on the path
        branches.append(longest)
        
        # remove longest branch
        for idx, e in enumerate(longest):
            if idx < len(longest) - 1:
                G_cp.remove_edge(longest[idx], longest[idx+1])
    
    # convert branches into list of paths
    paths = []
    for branch in branches:
        # get vertices in branch as n by 3 numpy.array, where n is length of branches
        path = np.zeros((len(branch),3), dtype=np.int64)
        for idx, node in enumerate(branch):
            path[idx,0] = np.int64(G_cp.nodes[node]['x'])
            path[idx,1] = np.int64(G_cp.nodes[node]['y'])
            path[idx,2] = np.int64(G_cp.nodes[node]['z'])
        
        paths.append(path)
    
    
    
#     if len(paths) == 1:
#         return np.array(paths[0])
#     else:
#         return np.array(*paths)

    return paths

In [5]:
## show conensus neuron G-002
# parameters
consen_neuron_path = '2018-08-01_G-002_consensus.swc'
spacing = np.array([0.29875923,0.3044159,0.98840415])
origin = np.array([70093.276,15071.596,29306.737])

# read swc into dataframe
df,_,_,_ = read_swc_offset(consen_neuron_path)
df.head()

,sample,structure,x,y,z,r,parent
0,1,0,73940.221323,18869.828297,33732.256716,1.0,-1
1,2,0,73942.312638,18858.869325,33745.105970,1.0,1
2,3,0,73942.312638,18833.907221,33732.256716,1.0,2
3,4,0,73945.897749,18822.339417,33720.395867,1.0,3
4,5,0,73939.325046,18822.948249,33705.569805,1.0,4


In [6]:
# convert from spatial to voxel coordinates
df_vox = swc_to_voxel(df,spacing,origin)
df_vox.head()

,sample,structure,x,y,z,r,parent
0,1,0,12876,12477,4477,1.0,-1
1,2,0,12883,12441,4490,1.0,1
2,3,0,12883,12359,4477,1.0,2
3,4,0,12895,12321,4465,1.0,3
4,5,0,12873,12323,4450,1.0,4


In [7]:
# convert neuron to graph
G = df_to_graph(df_vox)
len(df), len(G.nodes)

(1650, 1650)

In [8]:
# convert neuron graph into napari compatible format
paths_consen = graph_to_paths(G)

In [9]:
# display on napari
viewer_consen_neuron = napari.Viewer(ndisplay=3)
viewer_consen_neuron.add_shapes(paths_consen, shape_type='path', edge_color='white', name='neuron')

<Shapes layer 'neuron' at 0x133f94a20>

![screenshot](G-002_consensus.png)

Screenshot of entire G-002 consensus neuron in napari. It has a minimum bounding box of (7386, 9932, 5383) voxels, which is too large to load all at once into napari

In [10]:
sx = max(df['x']) - min(df['x'])
sy = max(df['y']) - min(df['y'])
sz = max(df['z']) - min(df['z'])
sx,sy,sz

(7386, 9932, 5383)

In [11]:
## show "spaghetti" sub-neuron (top right of screenshot) and overlay on sub-image

# open sub-image
image_path = 'G-002_15312-4400-6448_15840-4800-6656.tif'
img_comp = io.imread(image_path)
img_comp = np.swapaxes(img_comp,0,2)

# get sub-neuron from consensus neuron
start = np.array([15312,4400,6448])
end = np.array([15840,4800,6656])

G_comp = get_sub_neuron(G, start, end)

# convert sub-neuron graph into paths for napari
paths_comp = graph_to_paths(G_comp)

# display sub-neuron and sub-image on napari
viewer_comp = napari.Viewer(ndisplay=3)
viewer_comp.add_shapes(paths_comp, shape_type='path', edge_color='blue', name='sub-neuron')
viewer_comp.add_image(img_comp)

<Image layer 'Image' at 0x1481dae10>

![screenshot](G-002_complex.png)

Screenshot of sub-neuron G-002, the complex "spaghetti" portion

In [12]:
## show simple "curve" sub-neuron and overlay on sub-image

# open sub-image
image_path = 'G-002_15312-6400-5824_15840-6800-6032.tif'
img_simp = io.imread(image_path)
img_simp = np.swapaxes(img_simp,0,2)

# get sub-neuron from consensus neuron
start = np.array([15312,6400,5824])
end = np.array([15840,6800,6032])

G_simp = get_sub_neuron(G, start, end)

# convert sub-neuron graph into paths for napari
paths_simp = graph_to_paths(G_simp)

In [13]:
# display sub-neuron and sub-image on napari
viewer_simp = napari.Viewer(ndisplay=3)
viewer_simp.add_shapes(paths_simp, shape_type='path', edge_color='blue', name='sub-neuron')
viewer_simp.add_image(img_simp)

<Image layer 'Image' at 0x14a4bc470>

![screenshot](G-002_simple.png)

Screenshot of sub-neuron G-002, one of the simple "curve" portion

In [14]:
# path = np.array([[0, 0, 0], [0, 10, 10], [0, 5, 15], [20, 5, 15],
#         [56, 70, 21], [127, 127, 127]])

path = np.array([np.array([[0, 0, 0], [0, 10, 10], [0, 5, 15], [20, 5, 15],
        [56, 70, 21], [127, 127, 127]]),
        np.array([[0, 0, 0], [0, 10, 10], [0, 5, 15], [0, 5, 15],
            [0, 70, 21], [0, 127, 127]])])

path.shape

(2, 6, 3)

In [15]:
viewer = napari.Viewer(ndisplay=3)
p = viewer.add_shapes(path, shape_type='path', edge_width=4, edge_color=['red', 'red'])

In [16]:
# help(p)
p.edge_color

'black'

In [17]:
path2 = paths_consen
path3 = paths_comp
path4 = paths_simp

In [18]:
path

array([[[  0,   0,   0],
        [  0,  10,  10],
        [  0,   5,  15],
        [ 20,   5,  15],
        [ 56,  70,  21],
        [127, 127, 127]],

       [[  0,   0,   0],
        [  0,  10,  10],
        [  0,   5,  15],
        [  0,   5,  15],
        [  0,  70,  21],
        [  0, 127, 127]]])

In [19]:
type(path), type(path2), type(path3), type(path4)

(numpy.ndarray, list, list, list)

In [20]:
# path.shape, len(path2), len(path3), len(path4)
path.shape, path2.shape, path3.shape, path4.shape

AttributeError: 'list' object has no attribute 'shape'